<a href="https://colab.research.google.com/github/ammarsaf/aitinkerers-hackathon-supa-team-werecooked/blob/master/notebooks-benchmarking-exercises/benchmark_gemini_flash_llmasjudge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!wget https://raw.githubusercontent.com/wanadzhar913/aitinkerers-hackathon-supa-team-werecooked/refs/heads/master/datasets/for_presentation/boolq-eng-val-200.jsonl -q
!wget https://raw.githubusercontent.com/wanadzhar913/aitinkerers-hackathon-supa-team-werecooked/refs/heads/master/datasets/for_presentation/boolq-malay-val-200.jsonl -q
!wget https://raw.githubusercontent.com/wanadzhar913/aitinkerers-hackathon-supa-team-werecooked/refs/heads/master/datasets/for_presentation/fib-eng-val-200.jsonl -q
!wget https://raw.githubusercontent.com/wanadzhar913/aitinkerers-hackathon-supa-team-werecooked/refs/heads/master/datasets/for_presentation/fib-malay-val-200.jsonl -q

In [12]:
!pip install weave flash_attn accelerate bitsandbytes -U -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 39.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.7/310.7 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 5.0 MB/s eta 0:00:00


In [13]:
from google.colab import userdata
GEMINI_APIKEY = userdata.get('GEMINI_APIKEY')
WEAVE_APIKEY = userdata.get("WEAVE_APIKEY")

# Test

In [14]:
import google.generativeai as genai
import os

In [25]:
genai.configure(api_key=GEMINI_APIKEY)

# model_gem = genai.GenerativeModel(model_name="gemini-1.5-flash")
# response = model_gem.generate_content("Hello gemini who are you?")
# print(response.text)

# Dependencies

In [15]:
from glob import glob
import json
from tqdm.notebook import tqdm
import time
import seaborn as sns
import pandas as pd
import weave
import re
from typing import Dict

# GPU agnostic

In [6]:
!nvidia-smi

Mon Oct 21 03:16:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Load dataset

In [18]:
dataset_list = glob('*.jsonl')
dataset_list

['boolq-eng-val-200.jsonl',
 'fib-eng-val-200.jsonl',
 'boolq-malay-val-200.jsonl',
 'fib-malay-val-200.jsonl']

In [19]:
# construct Malay + English dataset
data_all = []

for k in dataset_list:
    with open(k) as fopen:
        for d in tqdm(fopen):
            d = json.loads(d)
            data_all.append(d)

print(f'Size of dataset: {len(data_all)}')

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

Size of dataset: 800


In [20]:
data_all[0]

{'question': 'does ethanol take more energy make that produces',
 'answer': 0,
 'passage': "All biomass goes through at least some of these steps: it needs to be grown, collected, dried, fermented, distilled, and burned. All of these steps require resources and an infrastructure. The total amount of energy input into the process compared to the energy released by burning the resulting ethanol fuel is known as the energy balance (or ``energy returned on energy invested''). Figures compiled in a 2007 report by National Geographic Magazine point to modest results for corn ethanol produced in the US: one unit of fossil-fuel energy is required to create 1.3 energy units from the resulting ethanol. The energy balance for sugarcane ethanol produced in Brazil is more favorable, with one unit of fossil-fuel energy required to create 8 from the ethanol. Energy balance estimates are not easily produced, thus numerous such reports have been generated that are contradictory. For instance, a separat

In [11]:
# construct Malay + English dataset
data_malay = []

for k in dataset_list:
    if 'malay' in k:
        with open(k) as fopen:
            for d in tqdm(fopen):
                d = json.loads(d)
                data_malay.append(d)

print(f'Size of dataset: {len(data_malay)}')

0it [00:00, ?it/s]

0it [00:00, ?it/s]

Size of dataset: 400


In [12]:
data_malay[0]

{'question': 'bolehkah anda memandu di kanada dengan lesen AS',
 'answer': 1,
 'passage': 'Orang yang memandu masuk ke Kanada mesti mempunyai dokumen pendaftaran kenderaan mereka dan bukti insurans.',
 'language': 'Malay'}

# Weave

In [13]:
PROJECT_NAME = 'benchmark_gemini-1.5-flash_llmasajudge_v1'

weave.init(PROJECT_NAME)

Please login to Weights & Biases (https://wandb.ai/) to continue:


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Logged in as Weights & Biases user: ammardev.
View Weave data at https://wandb.ai/ammardev/benchmark_gemini-1-5-flash_llmasajudge_v1/weave


# Benchmarking

In [1]:
def call_llm(message: str) -> str:
    """Function to call the LLM and generate output"""
    model_gem = genai.GenerativeModel(model_name="gemini-1.5-flash")
    response = model_gem.generate_content(message)
    return response.text

In [28]:
for test_data in data_all[:5]:
    print(call_llm(prompt_v1.format(passage=test_data["passage"], question=test_data["question"])))

{'consistency': 0} 

{'consistency': 1} 

{'consistency': 1} 

{'consistency': 1} 

{'consistency': 1} 



In [15]:
class GeminiFlashAsAJudge(weave.Model):
    prompt: str

    @weave.op
    def create_message(self, passage: str, question: str):
        return self.prompt.format(passage=passage, question=question)

    @weave.op
    def predict(self, passage:str, question:str):
        message = self.create_message(passage, question)
        return call_llm(message=message)

In [31]:
def accuracy(model_output, answer):
    try:
        model_output = json.loads(model_output)
        class_model_output = model_output.get('consistency', None)
    except json.JSONDecodeError:
        # to handle edge cases where the LLM outputs improper JSON like this: '1 {"consistency": 0'
        match = re.search(r'\"consistency\":\s*([01])', model_output)

        if match:
            number = match.group(1)
            class_model_output = int(number)
        else:
            class_model_output = None
    return {"accuracy": class_model_output == answer}


In [37]:
for test_data in data_all[:5]:
    model_output = call_llm(prompt_v1.format(passage=test_data["passage"], question=test_data["question"]))
    # print(model_output)
    acc = accuracy(model_output, test_data['answer'])
    print(acc)

{'accuracy': False}
{'accuracy': False}
{'accuracy': False}
{'accuracy': False}
{'accuracy': False}


In [17]:

class BinaryMetrics(weave.Scorer):
    class_name: str
    eps: float = 1e-8

    @weave.op()
    def summarize(self, score_rows) -> dict:
        # filter out None rows, model may error out sometimes...
        score_rows = [score for score in score_rows if score["correct"] is not None]
        # Compute f1, precision, recall
        tp = sum([not score["negative"] and score["correct"] for score in score_rows])
        fp = sum([not score["negative"] and not score["correct"] for score in score_rows])
        fn = sum([score["negative"] and not score["correct"] for score in score_rows])
        precision = tp / (tp + fp + self.eps)
        recall = tp / (tp + fn + self.eps)
        f1 = 2 * precision * recall / (precision + recall + self.eps)
        result = {"f1": f1, "precision": precision, "recall": recall}
        return result

    @weave.op()
    def score(self, answer: dict, model_output: dict|str) -> dict:
        try:
            model_output = json.loads(model_output)
            class_model_output = model_output.get(self.class_name, None)
        except json.JSONDecodeError:
            # to handle edge cases where the LLM outputs improper JSON like this: '1 {"consistency": 1'
            match = re.search(r'\"consistency\":\s*([01])', model_output)

            if match:
                number = match.group(1)
                class_model_output = int(number)
            else:
                class_model_output = None
        result = {
            "correct": class_model_output == answer,
            "negative": not class_model_output,
        }
        return result

F1 = BinaryMetrics(class_name="consistency")

## Prompt 1

In [22]:
prompt_v1 = """Anda adalah pakar dalam mengesan ketidakkonsistenan fakta dan halusinasi. Anda akan diberi satu dokumen dan satu soalan. Baca
dokumen dan soalan/kenyataan yang diberikan dengan teliti dan kenal pasti Ketidakkonsistenan Fakta (iaitu mana-mana soalan/kenyataan yang
tidak disokong atau bercanggah dengan maklumat dalam dokumen).

### Anda perlu memilih antara dua pilihan berikut:
- Tidak Konsisten dengan Fakta: Jika mana-mana soalan/kenyataan tidak disokong, terjawab atau bercanggah dengan dokumen, labelkannya sebagai 0.
- Konsisten dengan Fakta: Jika semua soalan/kenyataan disokong/terjawab oleh dokumen, labelkannya sebagai 1.

### Sebagai contoh:
Dokumen: "Gajah adalah mamalia besar yang biasanya ditemui di Afrika dan Asia. Mereka hidup dalam kumpulan yang dikenali sebagai kawanan dan terkenal kerana mempunyai ingatan yang baik."

Soalan/Kenyataan: "Gajah adalah mamalia besar yang biasanya ditemui di Eropah."
Jawapan: {{'consistency': 0}}

Soalan/Kenyataan: "Gajah adalah mamalia besar yang biasanya ditemui di Afrika dan Asia."
Jawapan: {{'consistency': 1}}

### Jawab berdasarkan dokumen dan soalan/kenyataan berikut:
Dokumen: {passage}
Soalan/Kenyataan: {question}

Kembalikan jawapan dalam format JSON untuk pilihan yang diberikan. Sebagai contoh: {{'consistency': 1}} atau {{'consistency': 0}}"""

In [19]:
geminiflash_asjudge = GeminiFlashAsAJudge(prompt=prompt_v1)

# Evaluate

## all data

In [20]:
evaluation_all = weave.Evaluation(dataset=data_all, scorers=[accuracy, F1])

In [21]:
await evaluation_all.evaluate(geminiflash_asjudge)

model_output failed

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/weave/flow/eval.py", line 191, in predict_and_score
    model_output, model_call = await async_call_op(
  File "/usr/local/lib/python3.10/dist-packages/weave/flow/eval.py", line 54, in async_call_op
    call_res = func.call(*args, __should_raise=True, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/weave/trace/op.py", line 344, in call
    return _do_call(
  File "/usr/local/lib/python3.10/dist-packages/weave/trace/op.py", line 393, in _do_call
    execute_result = _execute_call(
  File "/usr/local/lib/python3.10/dist-packages/weave/trace/op.py", line 309, in _execute_call
    handle_exception(e)
  File "/usr/local/lib/python3.10/dist-packages/weave/trace/op.py", line 307, in _execute_call
    res = func(*args, **kwargs)
  File "<ipython-input-15-9662f7da2ed0>", line 11, in predict
    return call_llm(message=message)
  File "<ipython-input-14-d582a2c57d5d>", line 4, in call_llm
    response = 

model_output failed

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/weave/flow/eval.py", line 191, in predict_and_score
    model_output, model_call = await async_call_op(
  File "/usr/local/lib/python3.10/dist-packages/weave/flow/eval.py", line 54, in async_call_op
    call_res = func.call(*args, __should_raise=True, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/weave/trace/op.py", line 344, in call
    return _do_call(
  File "/usr/local/lib/python3.10/dist-packages/weave/trace/op.py", line 393, in _do_call
    execute_result = _execute_call(
  File "/usr/local/lib/python3.10/dist-packages/weave/trace/op.py", line 309, in _execute_call
    handle_exception(e)
  File "/usr/local/lib/python3.10/dist-packages/weave/trace/op.py", line 307, in _execute_call
    res = func(*args, **kwargs)
  File "<ipython-input-15-9662f7da2ed0>", line 11, in predict
    return call_llm(message=message)
  File "<ipython-input-14-d582a2c57d5d>", line 4, in call_llm
    response = 

model_output failed

...

Evaluated 798 of 800 examples

Evaluated 799 of 800 examples

Evaluated 800 of 800 examples

🍩 https://wandb.ai/ammardev/benchmark_gemini-1-5-flash_llmasajudge_v1/r/call/0192ac9f-1967-7192-8cc7-e43ee622db72


KeyError: 'correct'